## Import Library 

In [1]:
!pip install tensorflow

In [5]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
enhancedretinopathy-dataset  model  retinopathy-dataset


In [8]:
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from tensorflow import lite
import tensorflow as tf
# load and evaluate a saved model
import numpy as np
from keras.models import load_model
from keras.preprocessing import image
from matplotlib import pyplot as plt
import cv2
import os
from google.colab.patches import cv2_imshow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn import preprocessing
from keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout
from sklearn.model_selection import train_test_split
import math
from keras.preprocessing import image
from PIL import Image
from matplotlib import cm

class Diabatic_Retino_Detection():
    def __init__(self, dataset_path="/content/drive/MyDrive/retinopathy-dataset",enhanceddataset_path="/content/drive/MyDrive/enhancedretinopathy-dataset",data_path="/content/drive/MyDrive/",models_path="'/content/drive/MyDrive/model/'"):
        self.dataset_path  = dataset_path
        self.enhanceddataset_path=enhanceddataset_path
        self.models_path=models_path
        self.data_path=data_path
        self.predictionsclasses = ['nosymptoms', 'symptoms']

    def image_acquisition(self,image_file, gray_scale=False):
        image_src= cv2.imread( image_file)
        if gray_scale:
          image_rgb = cv2.cvtColor(image_src, cv2.COLOR_BGR2GRAY)
        else:
          image_rgb = cv2.cvtColor(image_src, cv2.COLOR_BGR2RGB)
        TARGET_PIXEL_AREA = 100000.0
        img=image_rgb
        ratio = float(img.shape[1]) / float(img.shape[0])
        new_h = int(math.sqrt(TARGET_PIXEL_AREA / ratio) + 0.5)
        new_w = int((new_h * ratio) + 0.5)
        img2 = cv2.resize(img, (new_w,new_h))
        return img2

    def enhance_contrast(self,image_matrix, bins=256):
        image_flattened = image_matrix.flatten()
        image_hist = np.zeros(bins)

        # frequency count of each pixel
        for pix in image_matrix:
            image_hist[pix] += 1

        # cummulative sum
        cum_sum = np.cumsum(image_hist)
        norm = (cum_sum - cum_sum.min()) * 255
        # normalization of the pixel values
        n_ = cum_sum.max() - cum_sum.min()
        uniform_norm = norm / n_
        uniform_norm = uniform_norm.astype('int')

        # flat histogram
        image_eq = uniform_norm[image_flattened]
        # reshaping the flattened matrix to its original shape
        image_eq = np.reshape(a=image_eq, newshape=image_matrix.shape)

        return image_eq

    def equalize_this(self,image_file,saveimagepath, with_plot=False, gray_scale=False, bins=256):
        image_src = self.image_acquisition(image_file, gray_scale=False)
        if not gray_scale:
            r_image = image_src[:, :, 0]
            g_image = image_src[:, :, 1]
            b_image = image_src[:, :, 2]

            r_image_eq = self.enhance_contrast(image_matrix=r_image)
            g_image_eq = self.enhance_contrast(image_matrix=g_image)
            b_image_eq = self.enhance_contrast(image_matrix=b_image)

            image_eq = np.dstack(tup=(r_image_eq, g_image_eq, b_image_eq))
            cmap_val = None
        else:
            image_eq = self.enhance_contrast(image_matrix=image_src)
            cmap_val = 'gray'

        if with_plot:
            fig = plt.figure(figsize=(10, 20))

            ax1 = fig.add_subplot(2, 2, 1)
            ax1.axis("off")
            ax1.title.set_text('Original')
            ax2 = fig.add_subplot(2, 2, 2)
            ax2.axis("off")
            ax2.title.set_text("Equalized")

            ax1.imshow(image_src, cmap=cmap_val)
            ax2.imshow(image_eq, cmap=cmap_val)
            image_eq = Image.fromarray(np.uint8(cm.gist_earth(image_eq)*255))
            return True
        return image_eq
    def image_reshaping(self,image):
        #input_arr = image.img_to_array(image)
        #im = input_arr.resize((150, 150))
        # Convert single image to a batch.
        input_arr = image.img_to_array(image)
        input_arr = np.array([input_arr])
        #input_arr=np.asarray(image).astype(np.float32)
        #print(max(input_arr),min(input_arr))
        return input_arr

    def imagedata_preprocessing(self):
        print("Image Data preprocessing started...")
        print("Image Data acquisition started...")
        # preprocessing
        imgs_path = self.dataset_path
        #data = []
        #labels = []
        labelclasses = os.listdir(imgs_path)
        print("Image Data successfully acquisited.")
        print("Image Data enhancement started...")
        for labelclass in labelclasses:
            img_path = os.path.join(imgs_path, str(labelclass))  # 0-42
            for img in os.listdir(img_path):
                image_file=os.path.join(imgs_path, img)
                image_eq=self.equalize_this(image_file, with_plot=False, gray_scale=False, bins=256)
                im = self.image_reshaping(image_eq)
                data.append(im)
                labels.append(labelclass)
        print("Image Data successfully preprocessed")
        self.data = data
        self.labels = labels

    def imagedata_transformation(self):
        print("Image Data transformation started...")
        train_datagen = ImageDataGenerator(rescale=1./255,
                                      rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest')
        training_data = train_datagen.flow_from_directory(train_data_path, 
                                      target_size=(150, 150),
                                      batch_size=32,
                                      class_mode='binary') 
        for i in os.listdir("/content/drive/MyDrive/cottondataset/train/"):
          print(str(len(os.listdir("/content/drive/MyDrive/cottondataset/train/"+i))) +" "+ i +" images")
        print("Image Data successfully transformed.")
        self.training_data = training_data 

    def model_configration(self):
        print("Convolutional Model configration started...")
        model_path = self.models_path+'test.h5'
        checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
        callbacks_list = [checkpoint]
        model = keras.models.Sequential([
            keras.layers.Conv2D(filters=32, kernel_size=3, input_shape=[150, 150, 3]),
            keras.layers.MaxPooling2D(pool_size=(2, 2)),
            keras.layers.Conv2D(filters=64, kernel_size=3),
            keras.layers.MaxPooling2D(pool_size=(2, 2)),
            keras.layers.Conv2D(filters=128, kernel_size=3),
            keras.layers.MaxPooling2D(pool_size=(2, 2)),
            keras.layers.Conv2D(filters=256, kernel_size=3),
            keras.layers.MaxPooling2D(pool_size=(2, 2)),
            keras.layers.Dropout(0.5),
            keras.layers.Flatten(),
            keras.layers.Dense(units=128, activation='relu'),
            keras.layers.Dropout(0.1),
            keras.layers.Dense(units=256, activation='relu'),
            keras.layers.Dropout(0.25),
            keras.layers.Dense(units=2, activation='softmax')
        ])
        model.compile(optimizer=Adam(lr=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.summary()
        print("Convolutional Model successfully configured.")
        self.model=model
        self.callbacks_list= callbacks_list

    def train_model(self):
        print("Convolutional Model training started...")
        training_data=self.x_train
        callbacks_list=self.callbacks_list
        model= self.model
        history =model.fit(training_data, epochs=100, batch_size=16,
          validation_split=0.2, verbose=1, callbacks=callbacks_list)
        print("Convolutional Model successfully trained.")
        print("Convolutional Model Performance & testing results")
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()

        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()
        self.history=history
        
    def convert_hd5model_to_tflite(self):
        print("Convolutional Model conversion from to tflite started...")
        models = tf.keras.models.load_model('/content/drive/MyDrive/model/modelenhanced.hd5')
        converter = lite.TFLiteConverter.from_keras_model(models)
        print(converter)
        model = converter.convert()
        tfmodel_path=self.models_path+'model.tflite'
        assert isinstance(model, object)
        file = open(tfmodel_path, 'wb').write(model)
        print("Convolutional Model successfully converted from to tflite...")
        
    def test_model(self,Image_test_path):
        print("Image testing started...")
        # load model\
        model_path=self.models_path+'modelenhanced.hd5'
        model = load_model(model_path)
        # summarize model.
        # model.summary()
        # load dataset
        Image_test = Image_test_path
        #image1 = image.load_img(Image_test, target_size=(150, 150))
        # image1
        #input_arr = image.img_to_array(image1)
        a=self.equalize_this(Image_test , with_plot=False, gray_scale=False, bins=256)
        image1 = image.load_img('test.jpeg', target_size=(150, 150))
                #cv2_imshow( image)
        input_arr = self.image_reshaping(image1) # Convert single image to a batch.
        #input_arr
        predictions1 = model.predict(input_arr)
        print(predictions1)
        probabilities1 = model.predict_proba(input_arr)
        print(probabilities1)
        predictionsclasses = self.predictionsclasses
        probabilitiestest = list(probabilities1)
        testprob = max(probabilitiestest)
        print(testprob.tolist())
        max_value = max(testprob)
        print(max_value)
        for i in range(len(testprob)):
            if testprob[i] == max_value:
                max_index = i
                print(predictionsclasses[max_index])
        print("Image testing successfully completed.")


if __name__ == "__main__":
    Diabatic_Retino_Detection = Diabatic_Retino_Detection()
    Diabatic_Retino_Detection.imagedata_preprocessing()
    #Diabatic_Retino_Detection.imagedata_transformation()
    #Diabatic_Retino_Detection.model_configration()
    #Diabatic_Retino_Detection.train_model()
    #Image_test1 = '/content/drive/MyDrive/retinopathy-dataset/nosymptoms/10265_left.jpeg'
    #Image_test2 = '/content/drive/MyDrive/retinopathy-dataset/nosymptoms/10643_left.jpeg'
    #Image_test3 = '/content/drive/MyDrive/retinopathy-dataset/symptoms/10030_left.jpeg'
    #Image_test4 = '/content/drive/MyDrive/retinopathy-dataset/symptoms/10030_right.jpeg'
    #Diabatic_Retino_Detection.test_model(Image_test1)
    #Diabatic_Retino_Detection.test_model(Image_test2)
    #Diabatic_Retino_Detection.test_model(Image_test3)
    #Diabatic_Retino_Detection.test_model(Image_test4)

Image Data preprocessing started...
Image Data acquisition started...
Image Data successfully acquisited.
Image Data enhancement started...
/content/drive/MyDrive/enhancedretinopathy-dataset/symptoms/10297_right.jpeg
label symptoms filename 10297_right.jpeg
/content/drive/MyDrive/enhancedretinopathy-dataset/symptoms/10160_left.jpeg
label symptoms filename 10160_left.jpeg
/content/drive/MyDrive/enhancedretinopathy-dataset/symptoms/11159_left.jpeg
label symptoms filename 11159_left.jpeg
/content/drive/MyDrive/enhancedretinopathy-dataset/symptoms/11219_left.jpeg
label symptoms filename 11219_left.jpeg
/content/drive/MyDrive/enhancedretinopathy-dataset/symptoms/10220_left.jpeg
label symptoms filename 10220_left.jpeg
/content/drive/MyDrive/enhancedretinopathy-dataset/symptoms/11005_left.jpeg
label symptoms filename 11005_left.jpeg
/content/drive/MyDrive/enhancedretinopathy-dataset/symptoms/10694_left.jpeg
label symptoms filename 10694_left.jpeg
/content/drive/MyDrive/enhancedretinopathy-dat

KeyboardInterrupt: ignored

In [1]:
from matplotlib import pyplot as plt
import cv2
import numpy as np
from keras.preprocessing import image
from PIL import Image
import numpy as np
from PIL import Image
from matplotlib import cm
import math
# load a fundus image, normalized into 0-1 range (not 0-255)
# such as one from the IDRiD dataset  (assuming you already have it on disk)
#dset = IDRiD('./data/IDRiD_segmentation')
#img_id, img, labels = dset.sample()
#print("using image", img_id)
#import cv2

def read_this(image_file, gray_scale=False):
    image_src = cv2.imread(image_file)
    print(type(image_src))
    if gray_scale:
        image_rgb = cv2.cvtColor(image_src, cv2.COLOR_BGR2GRAY)
    else:
        image_rgb = cv2.cvtColor(image_src, cv2.COLOR_BGR2RGB)
    TARGET_PIXEL_AREA = 100000.0
    img=image_rgb
    ratio = float(img.shape[1]) / float(img.shape[0])
    new_h = int(math.sqrt(TARGET_PIXEL_AREA / ratio) + 0.5)
    new_w = int((new_h * ratio) + 0.5)
    img2 = cv2.resize(img, (new_w,new_h))
    print(type(img2))
    return img2
def enhance_contrast(image_matrix, bins=256):
    image_flattened = image_matrix.flatten()
    image_hist = np.zeros(bins)

    # frequency count of each pixel
    for pix in image_matrix:
        image_hist[pix] += 1

    # cummulative sum
    cum_sum = np.cumsum(image_hist)
    norm = (cum_sum - cum_sum.min()) * 255
    # normalization of the pixel values
    n_ = cum_sum.max() - cum_sum.min()
    uniform_norm = norm / n_
    uniform_norm = uniform_norm.astype('int')

    # flat histogram
    image_eq = uniform_norm[image_flattened]
    # reshaping the flattened matrix to its original shape
    image_eq = np.reshape(a=image_eq, newshape=image_matrix.shape)
    print(type(image_eq))
    return image_eq

def equalize_this(image_file, with_plot=False, gray_scale=False, bins=256):
    image_src = read_this(image_file=image_file, gray_scale=gray_scale)
    if not gray_scale:
        r_image = image_src[:, :, 0]
        g_image = image_src[:, :, 1]
        b_image = image_src[:, :, 2]

        r_image_eq = enhance_contrast(image_matrix=r_image)
        g_image_eq = enhance_contrast(image_matrix=g_image)
        b_image_eq = enhance_contrast(image_matrix=b_image)

        image_eq = np.dstack(tup=(r_image_eq, g_image_eq, b_image_eq))
        cmap_val = None
    else:
        image_eq = enhance_contrast(image_matrix=image_src)
        cmap_val = 'gray'

    if with_plot:
        fig = plt.figure(figsize=(10, 20))

        ax1 = fig.add_subplot(2, 2, 1)
        ax1.axis("off")
        ax1.title.set_text('Original')
        ax2 = fig.add_subplot(2, 2, 2)
        ax2.axis("off")
        ax2.title.set_text("Equalized")

        ax1.imshow(image_src, cmap=cmap_val)
        plt.imsave('test.png', image_src, cmap=cmap_val)
        ax2.imshow(image_eq, cmap=cmap_val)
        plt.imsave('test.png', np.uint8(image_eq), cmap=cmap_val)
        image_eq = Image.fromarray(np.uint8(cm.gist_earth(image_eq)*255))
        print(image_eq)
        return True
    print(type(image_eq))
    return image_eq
a=equalize_this(image_file='/content/drive/MyDrive/retinopathy-dataset/nosymptoms/10265_left.jpeg', with_plot=True)
print(a)
path="C:\\Somewhere\\myproperdirectory\\"
#from skimage.io._plugins.pil_plugin import ndarray_to_pil, pil_to_ndarray
#im=ndarray_to_pil(a).convert("1")
#a.save("your_file.jpeg")
#data = np.asarray( img, dtype="int32" )
#img = Image.fromarray( np.asarray( np.clip(data,0,255), dtype="uint8"), "L" )
input_arr = image.img_to_array(a)
input_arr = np.array([input_arr])
print(type(input_arr))
input_arr=input_arr.flatten()
print(type(input_arr))

<class 'NoneType'>


error: ignored

In [10]:
from google.colab import files
#plt.savefig("/content/test.png")
files.upload()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>